In [1]:
# ! pip install transformers
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

import torchsummary as summary
from tqdm import tqdm

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df[df['content']==' ']

In [3]:
torch.cuda.empty_cache()

In [4]:
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [5]:
df=pd.read_csv('BERT_data.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)

In [6]:
df['cat_1_level_2']=df['cat_1_level_2']-1
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 6, 5, 7], dtype=int64)

In [7]:
df=df[~(df['cat_1_level_2']==6)]
df=df[~(df['cat_1_level_2']==7)]
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 5], dtype=int64)

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['cat_1_level_2']
        self.text=[tokenizer(text,
                               padding='max_length',truncation=True,
                                return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [9]:
df

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,re confidenti employe informationlenhart,also need know base salari jay reitmey moniqu ...,0
1,<21041312.1075855725847.JavaMail.evans@thyme>,re person and confidenti compens inform,thank inform would help would send detail work...,2
2,<21261996.1075858638025.JavaMail.evans@thyme>,re western wholesal activ gas power conf call ...,guy coordin make sure someon listen confer cal...,3
3,<20399547.1075857614321.JavaMail.evans@thyme>,re confidenti employe informationdutch quigley,thx jeani slone pm john cc confidenti employe ...,4
4,<860767.1075849626951.JavaMail.evans@thyme>,re hp confidenti intern document,excel updat thank put togeth dalepatrick let r...,0
...,...,...,...,...
1485,<18361957.1075861368310.JavaMail.evans@thyme>,confidenti due dilig info g,lot confidenti inform item detail month pl cov...,0
1486,<29049822.1075842045435.JavaMail.evans@thyme>,california dim bulb,may seen first came back worth rerun,0
1487,<205897.1075861997314.JavaMail.evans@thyme>,re confidenti ee info,mark email fax ani written document give heidi...,4
1488,<23743848.1075863311776.JavaMail.evans@thyme>,confidenti inform and secur trade,william william enron wholesal servic offic ch...,0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df[['message_id','subject','content']], df['cat_1_level_2'], test_size=0.4, stratify=df['cat_1_level_2'])

In [11]:
X_train['cat_1_level_2']=y_train
X_test['cat_1_level_2']=y_test
df_train=X_train
df_val=X_test
df_test=0

In [12]:
'''
from sklearn.utils import resample
for i in [1,2,3,4,5,6]:
    spam_upsample = resample(df_train[df_train['cat_1_level_2']==i],
             replace=True,
             n_samples=len(df_train[df_train['cat_1_level_2']==0]) - len(df_train[df_train['cat_1_level_2']==i]),
             random_state=42)
    df_train=pd.concat([df_train,spam_upsample])
df_train.shape
'''

"\nfrom sklearn.utils import resample\nfor i in [1,2,3,4,5,6]:\n    spam_upsample = resample(df_train[df_train['cat_1_level_2']==i],\n             replace=True,\n             n_samples=len(df_train[df_train['cat_1_level_2']==0]) - len(df_train[df_train['cat_1_level_2']==i]),\n             random_state=42)\n    df_train=pd.concat([df_train,spam_upsample])\ndf_train.shape\n"

In [13]:
df_train.reset_index(inplace=True)
df_train=df_train.drop('index',axis=1)

df_val.reset_index(inplace=True)
df_val=df_val.drop('index',axis=1)
'''
df_test.reset_index(inplace=True)
df_test=df_test.drop('index',axis=1)
'''
print(len(df_train),len(df_val))

874 584


In [14]:
'''
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)
'''

'\nmodel_name = "bert-base-uncased"\nmodel = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)\n'

In [15]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 6)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [17]:
EPOCHS = 300
model = BertClassifier()
LR = 0.000001

In [18]:
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

201

In [19]:
# freeze first 7 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [20]:
'''
train, val = Dataset(df_train), Dataset(df_val)
train_dataloader = torch.utils.data.DataLoader(train, batch_size=20)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=20)
'''

'\ntrain, val = Dataset(df_train), Dataset(df_val)\ntrain_dataloader = torch.utils.data.DataLoader(train, batch_size=20)\nval_dataloader = torch.utils.data.DataLoader(val, batch_size=20)\n'

In [21]:
# train[111][0]['token_type_ids'][0]

In [22]:
'''
train, val = Dataset(df_train), Dataset(df_val)

model = model.cuda()
train_dataloader = torch.utils.data.DataLoader(train, batch_size=1)
for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)


                # output = model(input_id, mask)
                print(mask)
'''

"\ntrain, val = Dataset(df_train), Dataset(df_val)\n\nmodel = model.cuda()\ntrain_dataloader = torch.utils.data.DataLoader(train, batch_size=1)\nfor train_input, train_label in tqdm(train_dataloader):\n                train_label = train_label.to(device)\n                mask = train_input['attention_mask'].to(device)\n                input_id = train_input['input_ids'].squeeze(1).to(device)\n\n\n                # output = model(input_id, mask)\n                print(mask)\n"

In [23]:
# mask.shape

In [24]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=20)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=20)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    # optimizer = Adam(model.parameters(), lr= learning_rate,weight_decay=0.001) # for L2 penalty

    # for l1  penalty
    
    def L1_reg(l1_lambda):
        l1_reg=0
        for p in model.parameters():
            l1_reg=l1_reg+torch.abs(p).sum()
        return l1_lambda*l1_reg
        
    
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)


                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                total_loss_train=total_loss_train+L1_reg(0.00001)

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



train(model, df_train, df_val, LR, EPOCHS)

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:41<00:00,  1.06it/s]


Epochs: 1 | Train Loss:  1.906                 | Train Accuracy:  0.101                 | Val Loss:  0.100                 | Val Accuracy:  0.166


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:38<00:00,  1.13it/s]


Epochs: 2 | Train Loss:  1.887                 | Train Accuracy:  0.267                 | Val Loss:  0.087                 | Val Accuracy:  0.339


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:39<00:00,  1.12it/s]


...

Epochs: 298 | Train Loss:  1.807                 | Train Accuracy:  0.922                 | Val Loss:  0.092                 | Val Accuracy:  0.580


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:38<00:00,  1.13it/s]


Epochs: 299 | Train Loss:  1.807                 | Train Accuracy:  0.921                 | Val Loss:  0.092                 | Val Accuracy:  0.582


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:38<00:00,  1.14it/s]


Epochs: 300 | Train Loss:  1.807                 | Train Accuracy:  0.922                 | Val Loss:  0.090                 | Val Accuracy:  0.580


In [25]:
'''
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)
'''

'\ndef evaluate(model, test_data):\n\n    test = Dataset(test_data)\n\n    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)\n\n    use_cuda = torch.cuda.is_available()\n    device = torch.device("cuda" if use_cuda else "cpu")\n\n    if use_cuda:\n\n        model = model.cuda()\n\n    total_acc_test = 0\n    with torch.no_grad():\n\n        for test_input, test_label in test_dataloader:\n\n              test_label = test_label.to(device)\n              mask = test_input[\'attention_mask\'].to(device)\n              input_id = test_input[\'input_ids\'].squeeze(1).to(device)\n\n              output = model(input_id, mask)\n\n              acc = (output.argmax(dim=1) == test_label).sum().item()\n              total_acc_test += acc\n\n    print(f\'Test Accuracy: {total_acc_test / len(test_data): .3f}\')\n\nevaluate(model, df_test)\n'

In [26]:
torch.save(model.state_dict(), 'BERT_model_freeze.pth')